In [1]:
import numpy as np
import pandas as pd
import scipy
import sklearn.metrics
import os

import matplotlib.pyplot as plt
import matplotlib as mpl
#import seaborn as sns

from sklearn.metrics import roc_auc_score
import statsmodels.stats.multicomp as mc

# Collect predictions

In [7]:
regressor = 'MLP'
merge_embeddings = 1

In [8]:
data_dir = f'/lustre/groups/epigenereg01/workspace/projects/vale/mlm/variants/predictions/merge_embeddings_{merge_embeddings}/{regressor}/'

In [12]:
#models = ['Species-aware','Species-agnostic','DNABERT','DNABERT-2','NT-MS-v2-500M','4mers','5mers','word2vec','effective_length']
models = ['dnabert','dnabert-3utr-2e',  'dnabert2','dnabert2-zoo', 'dnabert2-3utr-2e', 'ntrans-v2-100m',  'ntrans-v2-100m-3utr-2e', 'stspace-3utr-2e', 'stspace-3utr-DNA', 'stspace-spaw-3utr-2e',
          'stspace-3utr-hs','5mers', 'effective_length', ]

In [17]:
res = {}

for subset in ('clinvar', 'gnomAD','eQTL-susie','CADD'):
        res[subset] = []
        for model in models:
            res_tsv = data_dir + f'{subset}-{model}.tsv'
            if os.path.isfile(res_tsv):
                df = pd.read_csv(res_tsv, sep='\t')
                df['model'] = model
                res[subset].append(df)
        if len(res[subset])>0:
            res[subset] = pd.concat(res[subset])
            N=res[subset].groupby('model').size().mean()
            print(subset)

clinvar
gnomAD
eQTL-susie
CADD


In [18]:
preds_res = {}
best_models = {}


for subset in ('clinvar', 'gnomAD','eQTL-susie','CADD'):

        if len(res[subset])>0:
        
            preds_res[subset] = res[subset].groupby('model').apply(lambda x:roc_auc_score(x.label.values,x.y_pred.values))
        
preds_res = pd.DataFrame(preds_res).map(lambda x: f'{x[0]:.2f}±{x[1]:.2f}' if type(x)==tuple else x)

preds_res

/tmp/ipykernel_1102768/3352880854.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  preds_res[subset] = res[subset].groupby('model').apply(lambda x:roc_auc_score(x.label.values,x.y_pred.values))
/tmp/ipykernel_1102768/3352880854.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  preds_res[subset] = res[subset].groupby('model').apply(lambda x:roc_auc_score(x.label.values,x.y_pred.values))
/tmp/ipykernel_1

,clinvar,gnomAD,eQTL-susie,CADD
model,,,,
dnabert,0.934682,0.582782,0.570608,0.601564
dnabert-3utr-2e,0.948272,0.595323,0.627507,0.583747
dnabert2,0.952061,0.564615,0.789081,0.537987
dnabert2-3utr-2e,0.954795,0.590034,0.821478,0.550759
dnabert2-zoo,0.954694,0.588238,0.831282,0.537035
ntrans-v2-100m,0.957902,0.570417,0.705879,0.527313
ntrans-v2-100m-3utr-2e,0.938498,0.576383,0.700542,0.545090
stspace-3utr-2e,0.928032,0.571889,0.778317,0.559495
stspace-3utr-DNA,0.919555,0.573063,0.751494,0.542675
